# Задание 4
- Шеверев Сергей Вячеславович
- 22М-05ММ
- Вариант 3: зависимая переменная $\texttt{irritability.1}$, остальные переменные будем считать независимыми.
    Независимые переменные упорядочим по значимости влияния на зависимую переменную, приведем соответсвующие частоты, значимости критерия хи-квадрат и точного критерия Фишера.
    
    По совместному распределению первых двух наиболее значимых переменных вычислим односторонний коэффициент неопределенности относительно зависимой переменной.
    

In [211]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats

In [212]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

Считаем датасет:

In [213]:
df = pd.read_csv('data_big.csv')

In [214]:
len(df.keys())

121

В датасете 120 признаков, один из которых - это порядковый номер записи. Нам нужны наблюдения в первый день и только  качественные признаки. Поэтому отфильтруем ненужные.

In [215]:
df.head()

,Unnamed: 0,X.1,depressed.mood.1,anxiety.1,suspiciousness.1,irritability.1,craving.to.alcohol.1,weakness.1,insomia.1,headache.1,...,combined2.9,HR.9,SBP.9,DBP.9,MBP.9,SV.9,CO.9,SI.9,CI.9,TPR.9
0,1,1,1,1,0,1,1,1,1,0,...,0,68.0,108.0,70.0,83.0,74.0,5.0,40.0,2.7,1321.0
1,2,2,1,1,0,0,1,1,2,1,...,9,63.0,114.0,70.0,85.0,123.0,7.7,60.0,3.8,879.0
2,3,3,1,1,0,0,0,2,1,0,...,0,64.0,120.0,80.0,93.0,106.0,6.8,55.0,3.5,1098.0
3,4,4,2,2,0,0,0,2,0,0,...,0,56.0,124.0,90.0,101.0,90.0,5.1,43.0,2.4,1600.0
4,5,5,1,1,0,0,2,2,1,0,...,0,66.0,116.0,78.0,90.0,90.0,5.9,45.0,2.2,1228.0


In [216]:
df = df [df.keys()[2:20]]

In [217]:
df.isnull().sum()

depressed.mood.1              0
anxiety.1                     0
suspiciousness.1              0
irritability.1                0
craving.to.alcohol.1          0
weakness.1                    0
insomia.1                     0
headache.1                    0
tremor.1                      0
polyuria.1                    0
sweating.1                    0
transient.hallusinations.1    0
vomiting.1                    0
anoreksia.1                   0
diarrhea.1                    0
thirst.1                      0
chest.pain.1                  0
hyperemia.1                   0
dtype: int64

Пропущенных значений нет - хорошо

Определим вспомогательные функции:
 - `entropy` - функция для подсчета энтропии
 - `conditional_probs` - функция для подсчета условных вероятностей (возвращает список)
 - `chisquare` - функция теста хи-квадрат
 - `uncertain` - функция для вычисления коэффициентов неопределенности

In [233]:
def entropy(feature):
    e = 0
    tmp = {}
    for i in feature:
        tmp[i] = tmp.get(i, 0) + 1
    for i in tmp: 
        e += tmp[i]/len(feature)*np.log2(tmp[i]/len(feature))
    return -e


def conditional_probs(X, Y):
    ct = np.array(pd.crosstab(X, Y), dtype='float64')
    row_sums = sum(ct.T)
    for i in range(ct.shape[0]):
        ct[i] /= row_sums[i]
    return ct


#хи-квадрат 
def chisquare(X, Y):
    ct = np.array(pd.crosstab(X, Y))
    N = sum(sum(ct))#сначала сумма по столбцам, потом суммируем суммы по столбцам
    col_sums = sum(ct)
    row_sums = sum(ct.T)
    chi2 = 0
    for i in range(ct.shape[0]):
        for k in range(ct.shape[1]):
            chi2 += np.power(ct[i][k],  2)/row_sums[i]/col_sums[k]
    df = (ct.shape[0] - 1)*(ct.shape[1] - 1)
    return 1 - scipy.stats.chi2.cdf(df = df, x=chi2)

def uncertain(X, Y = None , tab = None):
    ct = None
    if tab is None:
        ct = np.array(pd.crosstab(X, Y))
    else:
        ct = np.array(tab)
    XY_entropy = entropy(ct.flatten())
    X_entropy = entropy(sum(ct.T))
    Y_entropy = entropy(sum(ct))
    
    information = X_entropy + Y_entropy - XY_entropy
    return information/Y_entropy*100, information/X_entropy*100, information/(Y_entropy+X_entropy)*100

Посчитаем
 - энтропию каждого признака
 - таблицу условных вероятностей
 - точный критерий Пирсона
 - точный критерий Фишера
 - коэффициенты неопределенности

In [234]:
X = df['irritability.1']
uncertains = []
print(f'Энтропия признака irritability.1 {entropy(X)}\n\n')
for i in df.keys():
    if i == 'irritability.1':
        continue
    
    Y = df[i]
    a = lambda x: 1 if x < 2 else 2
    X_ = X.apply(a)
    Y_ = Y.apply(a) 
    print(f'Признак {i}:')
    #print(pd.crosstab(X, Y))
    print('Условные вероятности')
    print(conditional_probs(X, Y))
    print(f'Точный критерий Пирсона: {chisquare(X, Y)}')

    fisher_tab = pd.crosstab(X_, Y_)
    if fisher_tab.shape == (2, 2):
        fp = scipy.stats.fisher_exact(fisher_tab)[1]
        print(f'Точный критерий Фишера:  {fp}' + (' > 0.05 => различия можно объянить случайностью' if fp > 0.05 else ' < 0.05 => различия нельзя объяснить случайностью'))
    print(f'Энтропия {i}: {entropy(Y)}')
    uncertains.append((uncertain(X, Y), i))
    print(f'Коэффициенты неопределенности: {uncertains[-1][0]}')
    
    print('\n\n')

Энтропия признака irritability.1 1.1092800210442264


Признак depressed.mood.1:
Условные вероятности
[[0.80952381 0.19047619]
 [0.91666667 0.08333333]
 [0.         1.        ]]
Точный критерий Пирсона: 0.5601348815193194
Точный критерий Фишера:  0.1764705882352941 > 0.05 => различия можно объянить случайностью
Энтропия depressed.mood.1: 0.6722948170756379
Коэффициенты неопределенности: (33.3333333333333, 21.03099178571523, 12.89509357448471)



Признак anxiety.1:
Условные вероятности
[[0.19047619 0.66666667 0.14285714]
 [0.         0.91666667 0.08333333]
 [1.         0.         0.        ]]
Точный критерий Пирсона: 0.8686237218673036
Точный критерий Фишера:  1.0 > 0.05 => различия можно объянить случайностью
Энтропия anxiety.1: 1.0961085701090179
Коэффициенты неопределенности: (47.35399452381016, 47.35399452381016, 23.67699726190508)



Признак suspiciousness.1:
Условные вероятности
[[0.95238095 0.04761905]
 [1.         0.        ]
 [1.         0.        ]]
Точный критерий Пирсона: 0.6

Два самых влиятельных признака:

In [235]:
t= sorted(uncertains, key=lambda x: x[0][0])
for i in t[-2:]:
    print(i[1])

suspiciousness.1
transient.hallusinations.1


Очень даже логично :)

Посмотрим на соотвествующие относительные частоты:

In [236]:
crosstab = pd.crosstab(df['irritability.1'], [ df['suspiciousness.1'] , df['transient.hallusinations.1']  ] )
crosstab

suspiciousness.1             0        1
transient.hallusinations.1   0  1  2  1
irritability.1                         
0                           18  1  1  1
1                           12  0  0  0
2                            1  0  0  0

Посмотрим на совместное распределение:

In [237]:
uncertain(df['irritability.1'], tab = crosstab)

(94.96447867369888, 48.60847125000337, 32.15144728749485)

Наличие признаков $ \texttt{suspiciousness.1} \text{ и } \texttt{transient.hallusinations.1}$ определеяет почти 95% информации о зависимом признаке $ \texttt{irritability.1}$ 